The code here is adapted from the brev.dev Mistral fine-tuning tutorial by Harper Caroll at

https://github.com/brevdev/notebooks/blob/main/mistral-finetune-own-data.ipynb

Install libraries

In [ ]:
# only if pytorch needs to be installed in your conda env
!conda install --yes pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia

In [1]:
!pip install -U bitsandbytes

In [2]:
!conda install transformers

Channels:
 - defaults
 - pytorch
 - nvidia
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [3]:
!pip install -q -U git+https://github.com/huggingface/peft.git

In [4]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [5]:
!conda install datasets

Channels:
 - defaults
 - pytorch
 - nvidia
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [6]:
!conda install --yes scipy

Channels:
 - defaults
 - pytorch
 - nvidia
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [7]:
!conda install ipywidgets

Channels:
 - defaults
 - pytorch
 - nvidia
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [8]:
!conda install matplotlib

Channels:
 - defaults
 - pytorch
 - nvidia
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [1]:
import random
import os
from typing import Tuple

In [2]:
import torch

In [3]:
print(torch.__version__, torch.__file__)

2.1.1 /home/oblaat/.conda/envs/jack_conda_env/lib/python3.11/site-packages/torch/__init__.py


In [6]:
# file paths where we are going to store our results
filename_template = "results_raw/{num1}_digit/{num2}_digit/t_{idx:02}.txt"

WARNING: you need at least 16GB CPU RAM for this step!

In [8]:
# load base model - mistralai/Mistral-7B-Instruct-v0.1 - using 4-bit quantization
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
# set up the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [11]:
# test base Mistral on an example
eval_prompt = " Answer the following:\n38 + 11\na:"

In [12]:
# Re-init the tokenizer so it doesn't add padding or eos token
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 Answer the following:
38 + 11
a: 49


Some functions to help with batch testing:

In [13]:
# Helper functions for generating random n-digit numbers
def generate_random_n_digit_number(n):
    """
    Generates a random n-digit number.

    :param n: Number of digits in the number.
    :return: A random n-digit number.
    """
    # Ensure the first digit is not zero
    first_digit = random.randint(1, 9)

    # Generate the remaining n-1 digits, which can include zero
    remaining_digits = [random.randint(0, 9) for _ in range(n - 1)]

    # Combine the digits to form the number
    digits = [first_digit] + remaining_digits
    return int(''.join(map(str, digits)))

def get_random_pair(used_pairs: set, i: int, j: int) -> Tuple[int, int]:
    """
    Get pair of random numbers
    
    :param used_pairs: A set of all pairs of numbers used so far
    :param i: length in digits of first number
    :param j: length in digits of second number
    :return: A tuple of 2 numbers
    """
    nums_used = True
    while nums_used:
        num1 = generate_random_n_digit_number(i)
        num2 = generate_random_n_digit_number(j)
        pair = (num1, num2)
        nums_used = pair in used_pairs
    used_pairs.add(pair)
    return pair

In [14]:
import re
import random
from typing import Tuple

def extract_numbers_from_string(content):
    """
    Extract the two operands and the result from answer string.
    
    :param content: response from the LLM as a string
    :return: A tuple with operand 1, operand 2, and addition result
    """
    # Regex patterns to match numbers in the specified format
    addition_pattern = r'(\d+)\s*\+\s*(\d+)'
    a_number_pattern = r'a:\s*(\d+)'
    
    # Find and extract numbers for the addition
    addition_match = re.search(addition_pattern, content)
    if addition_match:
        num1, num2 = addition_match.groups()
    else:
        num1, num2 = None, None

    # Find and extract the number following 'a:'
    a_number_match = re.search(a_number_pattern, content)
    if a_number_match:
        a_number = a_number_match.group(1)
    else:
        a_number = None

    return int(num1), int(num2), int(a_number)

def run_problem(num1, num2):
    """
    get result of asking the LLM what num1 + num2 is as a string
    """
    eval_prompt = f" Answer the following:\n{num1} + {num2}\na:"
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        response = tokenizer.decode(model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)[0], skip_special_tokens=True)
    return response

One example to make sure those functions work:

In [15]:
used_pairs = set()
num1, num2 = get_random_pair(used_pairs, 6, 4)
response = run_problem(num1, num2)
_, _, answer = extract_numbers_from_string(response)
print(num1, num2, answer, answer == (num1 + num2))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


136310 6328 142638 True


In [16]:
from collections import defaultdict

Now for the batch testing. We'll test for 4, 6, 8, and 12 digit LHS, and 1-12 digits RHS. 100 trials for each RHS digit length.

In [19]:
num_trials = 20  # for my first run, what is on the poster, I used 100 trials, using 20 for brevity here

In [20]:
# i - number of LHS digits we'll use
results = {}
for i in [4, 6, 8, 12]:
    # j - number of RHS digits - we'll go from 1 to 12
    results[i] = defaultdict(list)
    for j in range(1, 13):
        used_pairs = set()
        for k in range(num_trials):
            num1, num2 = get_random_pair(used_pairs, i, j)
            response = run_problem(num1, num2)
            _, _, answer = extract_numbers_from_string(response)
            is_correct = answer == (num1 + num2)
            results[i][j].append((num1, num2, answer, is_correct))
            if k % 10 == 0:
                print(num1, num2, answer, is_correct)
            filename = filename_template.format(num1=i, num2=j, idx=k)
            directory = os.path.dirname(filename)
            if not os.path.exists(directory):
                os.makedirs(directory)
            with open(filename, 'w') as f_out:
                f_out.write(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


6193 7 6200 True


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


3968 3 3971 True


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


269113900839 943627827632 221274173901 False


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


529541928992 279445379878 809087308870 False


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Now the results!

In [21]:
accuracies = defaultdict(list)
for i in [4, 6, 8, 12]:
    for j in range(1, 13):
        acc = sum([1 for r in results[i][j] if r[3]])
        acc = acc * (100 / num_trials)
        accuracies[i].append(acc)
        
for i in [4, 6, 8, 12]:
    print(i, ": ", accuracies[i])

4 :  [95.0, 60.0, 90.0, 80.0, 70.0, 30.0, 5.0, 20.0, 30.0, 20.0, 20.0, 20.0]
6 :  [100.0, 70.0, 15.0, 40.0, 50.0, 70.0, 5.0, 0.0, 0.0, 10.0, 0.0, 0.0]
8 :  [100.0, 85.0, 20.0, 45.0, 0.0, 20.0, 20.0, 35.0, 5.0, 0.0, 0.0, 0.0]
12 :  [100.0, 35.0, 10.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0]


NOTE: the first time I ran these trials I got the following results. These are what are shown on the chart on the poster:

4: 98 83 86 87 70 43 18 24 26 17 5 4

6: 93 73 31 54 53 58 12 2 0 3 1 1

8: 91 82 40 19 3 6 26 39 6 0 0 0

12: 83 49 23 3 0 0 0 0 0 0 1 5